### technews parser to mongodb

In [2]:
## Code Info
# Version: 2018-02-04
# Data info: technews web's BTC news
# Data source: download from https://technews.tw/?s=%E6%AF%94%E7%89%B9%E5%B9%A3

In [3]:
## Init the data to DB for the first time
import re
import pymongo
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import MongoClient
from dateutil.relativedelta import relativedelta

# parse function
def parse(url):
    
    # Parse info
    print('################################## parseing...', url)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")

    # Parse_detail article's links
    for article in soup.select("article"):
        link = article.a['href']
        parse_detail(link)
    
    # Set page-query-index-limit: 這網頁很奇怪, 會重複, 且無限下去。
    try:
        pre_page = soup.select('.nav-previous a')[0].attrs['href']  # 向前換頁
        if int(pre_page.split('/')[-2]) < 5:
            print('################################## change page to', pre_page)
            parse(pre_page)
        else:
            print('################################## page index larger than 30, end now')
    except:
        print("no page")

# parse_detail function
def parse_detail(url):
    
    # Parse info
    print('parse_detail...', url)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    
    try:
        # Info inside
        source = 'technews'
        title = soup.select('.entry-title a')[0].text
        link = soup.select('.entry-title a')[0].attrs['href']

        # Transfet date format
        raw_date = soup.select(".content span[class=\"body\"]")[1].text # ex: '2017 年 07 月 17 日 13:11 '
        year = int(raw_date.split('年')[0].strip())
        month = int(raw_date.split('月')[0].split('年')[1].strip())
        day = int(raw_date.split('月')[1].split('日')[0].strip())
        hour = int(raw_date.split('日')[1].strip().split(':')[0])
        minute = int(raw_date.split('日')[1].strip().split(':')[1])
        sfttime = datetime.datetime(year, month, day, hour, minute) # ex: '2017-07-17 13:11:00'
        date = str(sfttime)

        # Clean content
        content = soup.select(".indent")[0].text
        remove_list = ['\n', '\xa0', ' ', '\u3000', '\t']    
        for item in remove_list:
            content = re.sub(item, '', content)

        # Collect data into list
        title_list.append(title)
        link_list.append(link)
        source_list.append(source)
        date_list.append(date)
        content_list.append(content)
        print('parse_detail...Completed', url)
    except:
        print('parse_detail...Failed', url)


# start_url: technews
start_url = "https://technews.tw/?s=%E6%AF%94%E7%89%B9%E5%B9%A3"

# For collecting data
title_list = []
link_list = []
date_list = []
source_list = []
content_list = []

In [4]:
def parse_detail(url):
    
    # Parse info
    print('parse_detail...', url)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    try:
        # Info inside
        source = 'technews'
        title = soup.select('.entry-title a')[0].text
        link = soup.select('.entry-title a')[0].attrs['href']

        # Transfet date format
        raw_date = soup.select(".content span[class=\"body\"]")[1].text # ex: '2017 年 07 月 17 日 13:11 '
        year = int(raw_date.split('年')[0].strip())
        month = int(raw_date.split('月')[0].split('年')[1].strip())
        day = int(raw_date.split('月')[1].split('日')[0].strip())
        hour = int(raw_date.split('日')[1].strip().split(':')[0])
        minute = int(raw_date.split('日')[1].strip().split(':')[1])
        sfttime = datetime.datetime(year, month, day, hour, minute) # ex: '2017-07-17 13:11:00'
        date = str(sfttime)

        # Clean content
        content = soup.select(".indent")[0].text
        remove_list = ['\n', '\xa0', ' ', '\u3000', '\t']    
        for item in remove_list:
            content = re.sub(item, '', content)

        # Collect data into list
        title_list.append(title)
        link_list.append(link)
        source_list.append(source)
        date_list.append(date)
        content_list.append(content)
        print('parse_detail...Completed', url)
    except:
        print('parse_detail...Failed', url)

In [5]:
parse(start_url)

################################## parseing... https://technews.tw/?s=%E6%AF%94%E7%89%B9%E5%B9%A3
parse_detail... https://finance.technews.tw/2018/06/29/bitcoin-fell-below-6000-dollars/
parse_detail...Completed https://finance.technews.tw/2018/06/29/bitcoin-fell-below-6000-dollars/
parse_detail... https://finance.technews.tw/2018/06/26/kimchi-premium-disappear/
parse_detail...Completed https://finance.technews.tw/2018/06/26/kimchi-premium-disappear/
parse_detail... https://technews.tw/2018/06/25/goldman-sachs-ceo-bitcoin/
parse_detail...Completed https://technews.tw/2018/06/25/goldman-sachs-ceo-bitcoin/
parse_detail... https://technews.tw/2018/06/23/europes-first-bitcoin-atm-at-schiphol/
parse_detail...Completed https://technews.tw/2018/06/23/europes-first-bitcoin-atm-at-schiphol/
parse_detail... https://finance.technews.tw/2018/06/15/bitcoin-price-down-another-reason/
parse_detail...Completed https://finance.technews.tw/2018/06/15/bitcoin-price-down-another-reason/
parse_detail... htt

parse_detail...Completed https://technews.tw/2018/03/26/bitcoin-tax-finance-minister-taiwan/
parse_detail... https://technews.tw/2018/03/26/child-abuse-image-in-bitcoin-blockchain/
parse_detail...Completed https://technews.tw/2018/03/26/child-abuse-image-in-bitcoin-blockchain/
parse_detail... https://technews.tw/2018/03/23/bitcoin-falls-on-fears-of-regulatory-trouble-for-big-crypto-exchange/
parse_detail...Completed https://technews.tw/2018/03/23/bitcoin-falls-on-fears-of-regulatory-trouble-for-big-crypto-exchange/
################################## change page to https://technews.tw/page/4/?s=%E6%AF%94%E7%89%B9%E5%B9%A3
################################## parseing... https://technews.tw/page/4/?s=%E6%AF%94%E7%89%B9%E5%B9%A3
parse_detail... https://technews.tw/2018/03/21/blockchain-will-survive-even-if-bitcoin-does-not/
parse_detail...Completed https://technews.tw/2018/03/21/blockchain-will-survive-even-if-bitcoin-does-not/
parse_detail... https://technews.tw/2018/03/19/g20-bitcoin-pric

In [6]:
# Collect and Clean
all_dict = {'Title':title_list,
        'Source':source_list,
        'Link':link_list,
        'Date':date_list,
        'Content':content_list
       }
df = pd.DataFrame(all_dict) 

# Set time-date-format
for index, row in df.iterrows():
    try:
        x = datetime.datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S')
        df.at[index, 'Date'] = x
        
    except:
        print(index, 'error')
        
df.sort_values(by='Date')

,Title,Source,Link,Date,Content
59,偷不走比特幣的匪徒，偷走冰島 600 部挖礦電腦,technews,https://technews.tw/2018/03/07/600-mining-comp...,2018-03-07 08:15:00,冰島由於電力成本低，最近成了全球最熱門的「挖礦」聖地，不少希望挖掘加密貨幣的公司都會在當地設...
58,比特幣人氣減弱，成交量盪兩年新低,technews,https://finance.technews.tw/2018/03/07/bitcoin...,2018-03-07 09:30:00,"比特幣去年底一度逼近20,000美元，但自高點暴跌腰斬後，至今欲振乏力，原因可能與比特幣人氣..."
57,台灣遠東航空宣布接受比特幣支付,technews,https://technews.tw/2018/03/07/taiwan-far-east...,2018-03-07 10:00:00,台灣遠東航空近日宣布，將接受乘客以比特幣進行支付活動，用於機票和相關旅遊服務。目前，乘客可以...
56,政府管制來了，比特幣失守 1 萬美元大關,technews,https://finance.technews.tw/2018/03/08/bitcoin...,2018-03-08 14:00:00,政府監管大刀揮向虛擬貨幣市場，虛幣交易所和電子錢包業者都將須遵守證券法限制，虛幣聞訊全線跳水...
55,美日監管變嚴，比特幣單週狂瀉近四分之一,technews,https://technews.tw/2018/03/09/bitcoin-price-d...,2018-03-09 15:40:00,"主張去中央化的虛擬貨幣終究難逃政府監管，由於近期美日監管趨嚴，比特幣9日再度跌破9,000美..."
54,高盛：比特幣陷破底危機，恐再摔 35%,technews,https://finance.technews.tw/2018/03/13/goldman...,2018-03-13 12:00:00,比特幣上周再次失速暴跌後，高盛技術分析團隊看衰比特幣不僅尚未脫離險境，且可能持續往下跌破2月...
53,"挖礦一年耗掉 485 億度電，南韓挖一枚比特幣成本要 26,170 美元",technews,https://technews.tw/2018/03/13/digiconomist-cr...,2018-03-13 18:30:00,加密貨幣興起之後，挖礦一直讓各國政府很頭大。除了加密貨幣本身牽扯資安及洗錢問題，挖礦帶來的電...
52,用比特幣在用戶自動販賣機點餐，這家交易所準備這麼做,technews,https://technews.tw/2018/03/14/bithumb-to-brin...,2018-03-14 07:45:00,據Coinjournal報導，南韓最大加密貨幣交易所Bithumb已宣布計劃提供餐廳和咖啡館...
51,機器人有安全漏洞，對客人爆粗口和索取比特幣,technews,https://technews.tw/2018/03/14/pepper-nao-be-h...,2018-03-14 08:45:00,Pepper、NAO是日本SoftbankRobotics研發的人型機器人，在日本已展開商業...
50,比特幣摜破 8 千美元，摔至 2 月神祕買家平均成本區,technews,https://finance.technews.tw/2018/03/15/cryptos...,2018-03-15 16:55:00,比特幣跌勢煞不住，繼14日重挫10%後，週四亞洲盤又續跌逾6%，一口氣摜破8千美元整數關卡，...


In [7]:
_list = []
for index, row in df.iterrows():
    _ = {
        "Date": row['Date'],
        "Link": row['Link'],
        "Source": row['Source'],
        "Title": row['Title'],
        "Content": row['Content']
    }
    _list.append(_)
    
# send to db    
client = MongoClient('mongodb://localhost:27017/')
db = client.BTC_NEWS_DB
for item in _list:
    db.BTC_TW_NEWS_COLLECTION.insert_one(item)
print('Database Updated')


# import urllib.parse
# username = urllib.parse.quote_plus('root')
# password = urllib.parse.quote_plus('pecupecu520')
# client = MongoClient('mongodb://%s:%s@140.112.145.148:26666' % (username, password))
# db = client.BTC_NEWS_DB
# for item in tt:
#     db.BTC_TW_NEWS_COLLECTION.insert_one(item)
# print('Database Updated')

Database Updated
